<a href="https://colab.research.google.com/github/ritwiks9635/Object_Recognition_And_Detection-/blob/main/Aerial_Image_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Aerial Image Detection**

In [1]:
#https://www.kaggle.com/datasets/ammarnassanalhajali/mmdetectron-31-wheel

In [2]:
#https://www.kaggle.com/datasets/haoqingsong/frozen-packages-mmdetection

In [3]:
#https://www.kaggle.com/datasets/kailaspsudheer/tiny-object-detection

In [ ]:
!unzip /content/https:/www.kaggle.com/datasets/kailaspsudheer/tiny-object-detection/tiny-object-detection.zip

In [ ]:
!unzip /content/https:/www.kaggle.com/datasets/ammarnassanalhajali/mmdetectron-31-wheel/mmdetectron-31-wheel.zip

In [9]:
!unzip /content/https:/www.kaggle.com/datasets/haoqingsong/frozen-packages-mmdetection/frozen-packages-mmdetection.zip

In [10]:
import IPython.display as display
!pip install --no-index --no-deps /content/pycocotools-2.0.6-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /content/torch-1.12.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /content/torchvision-0.13.1+cu116-cp310-cp310-linux_x86_64.whl
!pip install --no-index --no-deps /content/mmcv-2.0.1-cp310-cp310-manylinux1_x86_64.whl
!pip install --no-index --no-deps /content/openmim-0.3.9-py2.py3-none-any.whl
!pip install --no-index --no-deps /content/mmengine-0.7.4-py3-none-any.whl
!pip install --no-index --no-deps /content/addict-2.4.0-py3-none-any.whl
!pip install yapf==0.40.1
display.clear_output()

In [ ]:
!git clone https://github.com/open-mmlab/mmdetection.git
%cd mmdetection
!pip install -v -e .

In [12]:
!mkdir checkpoint
!mkdir outputs
!mkdir data
!mkdir data/images
!mkdir data/labels

display.clear_output()

In [13]:
import torch, torchvision,mmdet
print("torch=",torch.__version__,torch.cuda.is_available())
print("mmdet=",mmdet.__version__)
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

torch= 1.12.1+cu116 True
mmdet= 3.3.0
11.6
GCC 9.3


In [14]:
import os
import io
import shutil
from PIL import Image

In [16]:
from pycocotools.coco import COCO

annot_path = "/content/SkyFusion/train/_annotations.coco.json"

coco = COCO(annot_path)

categories = coco.loadCats(coco.getCatIds())
category_id_to_name = {cat['id']: cat['name'] for cat in categories}


for category_id, category_name in category_id_to_name.items():
    print(f"Category ID: {category_id}, Category Name: {category_name}")

loading annotations into memory...
Done (t=0.40s)
creating index...
index created!
Category ID: 1, Category Name: Aircraft
Category ID: 2, Category Name: ship
Category ID: 3, Category Name: vehicle


In [17]:
!wget https://download.openmmlab.com/mmdetection/v3.0/ddq/ddq_detr_swinl_30e.pth -P ./checkpoints
display.clear_output()

In [18]:
from mmengine import Config
cfg = Config.fromfile('configs/ddq/ddq-detr-4scale_swinl_8xb2-30e_coco.py')

In [19]:
config_car = """
_base_ = './ddq-detr-4scale_swinl_8xb2-30e_coco.py'

data_root = '/content/SkyFusion/'

train_batch_size_per_gpu = 1
train_num_workers = 2

max_epochs = 20

metainfo = {
    'classes': ('Aircraft', 'ship', 'vehicle'),
    'palette': [
        (127, 201, 127), (190, 174, 212), (253, 192, 134),
    ]
}

train_pipeline = [
    dict(type='LoadImageFromFile', backend_args=_base_.backend_args),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(type='RandomFlip', prob=0.5),
    dict(
        type='RandomChoice',
        transforms=[
            [
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 1280)],
                    keep_ratio=True)
            ],
            [
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 4200)],
                    keep_ratio=True),
                dict(
                    type='RandomCrop',
                    crop_type='absolute_range',
                    crop_size=(384, 640),
                    allow_negative_crop=True),
                dict(
                    type='RandomChoiceResize',
                    scales=[(1280, 1280)],
                    keep_ratio=True)
            ]
        ]),
    dict(type='PackDetInputs')
]

test_pipeline = [
    dict(type='LoadImageFromFile', backend_args=None),
    dict(type='Resize', scale=(1280, 1280), keep_ratio=True),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='PackDetInputs',
        meta_keys=('img_id', 'img_path', 'ori_shape', 'img_shape',
                   'scale_factor'))
]

train_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='train/'),
        ann_file='train/_annotations.coco.json'))

val_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='valid/'),
        ann_file='valid/_annotations.coco.json'))

test_dataloader = dict(
    batch_size=train_batch_size_per_gpu,
    num_workers=train_num_workers,
    dataset=dict(
        data_root=data_root,
        metainfo=metainfo,
        data_prefix=dict(img='test/'),
        ann_file='test/_annotations.coco.json'))

val_evaluator = dict(ann_file=data_root + 'valid/_annotations.coco.json')

test_evaluator = dict(ann_file=data_root + 'test/_annotations.coco.json')

model = dict(backbone=dict(frozen_stages=4), bbox_head=dict(num_classes=3))

optim_wrapper = dict(
    type='OptimWrapper',
    optimizer=dict(
        type='AdamW',
        lr=0.0002/16,
        weight_decay=0.05),
    clip_grad=dict(max_norm=0.1/16, norm_type=2),
    paramwise_cfg=dict(custom_keys={'backbone': dict(lr_mult=0.1/16)})
)

default_hooks = dict(
    checkpoint=dict(
        interval=1,
        max_keep_ckpts=2,
        save_best='auto'
    ),
    logger=dict(type='LoggerHook', interval=2000))

load_from = './checkpoints/ddq_detr_swinl_30e.pth'

train_cfg = dict(type='EpochBasedTrainLoop', max_epochs=max_epochs, val_interval=2)
randomness = dict(seed=2023, deterministic=False, diff_rank_seed=False)
"""

with open('./configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py', 'w') as f:
    f.write(config_car)

In [ ]:
!python tools/train.py configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py

In [ ]:
from mmdet.apis import DetInferencer
import glob

config = 'configs/ddq/ddq-detr-4scale_swinl_1xb1-15e_sky.py'
checkpoint = glob.glob('./work_dirs/ddq-detr-4scale_swinl_1xb1-15e_sky/best_coco*.pth')[0]

device = 'cuda:0'

inferencer = DetInferencer(config, checkpoint, device)

img = '/content/SkyFusion/test/007b39d27_png_jpg.rf.f842b89874525c4347de23216da3cd92.jpg'
result = inferencer(img, out_dir='./output', pred_score_thr=0.6)

display.clear_output()
Image.open('./output/vis/007b39d27_png_jpg.rf.f842b89874525c4347de23216da3cd92.jpg')

In [ ]:
img = '/content/SkyFusion/test/014de911-7810-4f7d-8967-3e5402209f4a_0_0_jpg.rf.7b6e48f643ecb69e2a57f0dc685686ce.jpg'
result = inferencer(img, out_dir='./output', pred_score_thr=0.6)

display.clear_output()
Image.open('./output/vis/014de911-7810-4f7d-8967-3e5402209f4a_0_0_jpg.rf.7b6e48f643ecb69e2a57f0dc685686ce.jpg')